In [1]:
import math
import numpy as np
import pandas as pd
from scipy.stats import norm
from tqdm import tqdm

In [2]:
'''
1.Using parametric methods generating price of option.
  Tow parametric methods of option pricing:
  1.1 Black-Scholes model : bs_generate(S, K, T, r, sigma)
  1.2 Binomial tree method: bnt_generate(S, K, T, r, sigma)
'''

'\n1.Using parametric methods generating price of option.\n  Tow parametric methods of option pricing:\n  1.1 Black-Scholes model : bs_generate(S, K, T, r, sigma)\n  1.2 Binomial tree method: bnt_generate(S, K, T, r, sigma)\n'

In [2]:
# bs method
def bs_generate(S, K, T, r, sigma):
  '''
  S: spot price
  K: strike price
  T: time to maturity
  r: risk-free interest rate
  sigma: standard deviation of price of underlying asset
  '''
  d1 = ( np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)

  #  call option prcie
  C = (S * norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r*T) * norm.cdf(d2, 0.0, 1.0))

  return C

In [25]:
# binomial tree
def bnt_generate(N,T,S0,sigma,r,K,show_array=False):
  '''
  N: number of time interval (number of tree seperation)
  '''
  dt=T/N
  u=np.exp(sigma*np.sqrt(dt))
  d=1/u
  p=(np.exp(r*dt)-d)/(u-d)# 贴现

  price_tree=np.zeros([N+1,N+1])
  for i in range(N+1):
      for j in range(i+1):
          price_tree[j,i]=S0*(d**j)*(u**(i-j))
  # pick ITM
  option_tree=np.zeros([N+1,N+1])
  option_tree[:,N]=np.maximum(np.zeros(N+1),price_tree[:,N]-K)

  for i in np.arange(N-1,-1,-1): # [2,1,0]
      for j in np.arange(0,i+1):# [0,1,2]
          option_tree[j,i]=np.exp(-r*dt)*(p*option_tree[j,i+1]+(1-p)*option_tree[j+1,i+1])

  if show_array:
    return [option_tree[0,0],np.round(price_tree),np.round(option_tree)]
  else:
    return option_tree[0,0]

In [30]:
def mc_generate(self,T,S0,sigma,r,K):
        np.random.seed(522)
        random_Z =  np.random.randn(1000)
        
        def discount_ST(randZ):
            St = S0*np.exp((r-0.5*sigma**2)*T + randZ*sigma*np.sqrt(T))
            St_discount = np.exp(-r*T)*np.max(St-K,0)
            return St_discount

        STs = np.apply_along_axis(random_Z, discount_ST)
        C = np.nanmean(STs)
        return C

In [27]:
params = {
    'S':np.arange(1,101),
    'K':np.arange(1,101),
    'sigma':np.arange(1,51,10)/100,
    'T':np.array([0.25, 0.5, 0.75, 1.0, 1.5, 1, 5]),
    'r':np.arange(1,11)/100
}

# generating data
def generate_params_com(params_dict):
    generating_dict = {}
    for s in params['S']:
        for k in params['K']:
            for sig in params['sigma']:
                for t in params['T']:
                    for r in params['r']:
                        generating_dict[f'S{s}_K{k}_sigma{sig}_T{t}_r{r}'] = np.array([s,k,sig,t,r])

In [31]:
generating_dict = {}
for s in tqdm(params['S']):
    for k in params['K']:
        for sig in params['sigma']:
            for t in params['T']:
                for r in params['r']:
                    generating_dict[f'S{s}_K{k}_sigma{sig}_T{t}_r{r}'] = np.array([s,k,sig,t,r,bs_generate(s, k, t, r, sig),binomial_tree_call_option_pricing_model(10,t,s,sig,r,k,show_array=False)])

100%|██████████| 100/100 [58:21<00:00, 35.01s/it]


In [1]:
import pickle
import pandas as pd

In [2]:
d = pd.read_pickle('distilled_data.pkl')

In [3]:
d

,T,S0,sigma,r,K,BS,BNT,MC
S1_K1_sigma0.01_T0.25_r0.01,0.25,1.0,0.01,0.01,1.0,0.003485,0.003430,0.017336
S1_K1_sigma0.01_T0.25_r0.02,0.25,1.0,0.01,0.02,1.0,0.005403,0.005344,0.019826
S1_K1_sigma0.01_T0.25_r0.03,0.25,1.0,0.01,0.03,1.0,0.007618,0.007568,0.022311
S1_K1_sigma0.01_T0.25_r0.04,0.25,1.0,0.01,0.04,1.0,0.009992,0.009965,0.024789
S1_K1_sigma0.01_T0.25_r0.05,0.25,1.0,0.01,0.05,1.0,0.012432,0.012423,0.027261
...,...,...,...,...,...,...,...,...
S100_K100_sigma0.41_T5.0_r0.06,5.00,100.0,0.41,0.06,100.0,45.220078,44.440443,906.364640
S100_K100_sigma0.41_T5.0_r0.07,5.00,100.0,0.41,0.07,100.0,46.876793,46.106417,909.977653
S100_K100_sigma0.41_T5.0_r0.08,5.00,100.0,0.41,0.08,100.0,48.527247,47.765452,913.414458
S100_K100_sigma0.41_T5.0_r0.09,5.00,100.0,0.41,0.09,100.0,50.168302,49.414757,916.683647


In [34]:
df = pd.DataFrame(columns=['s', 'k', 't', 'r', 'sig','bs','bnt'])
df = df.append(generating_dict, ignore_index=True)

In [8]:
df = pd.DataFrame(columns=['s', 'k', 't', 'r', 'sig','bs','bnt'], data = [['a',4,3,4,5,6,7],['a',2,3,4,5,6,7],['b',2,3,4,5,6,7]])

In [9]:
df

,s,k,t,r,sig,bs,bnt
0,a,4,3,4,5,6,7
1,a,2,3,4,5,6,7
2,b,2,3,4,5,6,7
